# Decision Trees
## Exercise 1 -- Download and load the dataset

In [5]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])

X = data.loc[:, data.columns != 'class'].values
y = data['class'].values

In [6]:
X[0]

array([ 3.6216 ,  8.6661 , -2.8073 , -0.44699])

In [7]:
income_data = pd.read_csv("adult.data", 
        names=["age", "workclass","fnlwgt","education","education-num","marital-status","occupation",
                "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"], index_col=False)

binary_income_data = pd.get_dummies(income_data).drop("income_ <=50K", axis=1)

X = binary_income_data.drop("income_ >50K", axis=1).values
y = binary_income_data["income_ >50K"].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Exercise 2 -- Gini Index

In [9]:
def gini_index(region):
    """
    Implements the gini index over the classes in a region
    
    Parameters
    ----------
    region : array (1D)
        array of labels assigned to points in this region
        
    Returns
    -------
    float
        the Gini Index
    """
    classes = np.unique(region)
    N = region.shape[0]
    gini = 0
    for c in classes:
        p = (region == c).sum() / N
        gini += p*(1-p)
    return gini

In [10]:
# test your code
np.random.seed(0)
gini_index(np.random.randint(0, 2, 20)), gini_index(np.ones(10)), gini_index(np.zeros(10)), gini_index(np.array([]))

(0.495, 0.0, 0.0, 0)

## Exercise 3 -- Make a split

In [11]:
def split_region(region, feature_index, tau):
    """
    Given a region, splits it based on the feature indicated by
    `feature_index`, the region will be split in two, where
    one side will contain all points with the feature with values 
    lower than `tau`, and the other split will contain the 
    remaining datapoints.
    
    Parameters
    ----------
    region : array of size (n_samples, n_features)
        a partition of the dataset (or the full dataset) to be split
    feature_index : int
        the index of the feature used to make this partition
    tau : float
        The threshold used to make this partition
        
    Return
    ------
    low_partition : array
        indices of the datapoints in `region` where feature < `tau`
    high_partition : array
        indices of the datapoints in `region` where feature < `tau` 
    """
    return np.where(region[:, feature_index] < tau)[0], np.where(region[:, feature_index] >= tau)[0]

In [12]:
r, l = split_region(X_train, 1, 0)
print(r.shape, l.shape)

(0,) (22792,)


## Exercise 4 -- Find the best split

In [13]:
def get_split(X, y):
    best_gini = 100 # unreasonably high Gini Index
    best_feature_index = None
    best_tau = None
    best_lo = None
    best_hi = None
    for feature_index in range(X.shape[1]):
        for tau in X[:, feature_index]:
            lo, hi = split_region(X, feature_index, tau)
            gini = gini_index(y[lo]) + gini_index(y[hi])
            if gini < best_gini:
                best_gini = gini
                best_feature_index = feature_index
                best_tau = tau
                best_lo = lo
                best_hi = hi
    return {
        'feature_index': best_feature_index,
        'tau': best_tau,
        'low_region' : best_lo,
        'high_region' : best_hi,
    }

In [14]:
get_split(X_train[:2, :], y_train[:2])

{'feature_index': 0,
 'tau': 47,
 'low_region': array([0], dtype=int64),
 'high_region': array([1], dtype=int64)}

## Exercise 5 -- Recursive Splitting

In [15]:
def recursive_growth(node, min_samples, max_depth, current_depth, X, y):
    """
    Recursively grows a decision tree.
    
    Parameters
    ----------
    node : dictionary
        If the node is terminal, it contains only the "class" key, which determines the value to be used as a prediction.
        If the node is not terminal, the dictionary has the structure defined by `get_split`
    min_samples : int
        parameter for stopping criterion
    max_depth : int
        parameter for stopping criterion
    depth : int
        current distance from the root
    X : array (n_samples, n_features)
        features (full dataset)
    y : array (n_samples, )
        labels (full dataset)
    
    Notes
    -----
    To create a terminal node, a dictionary is created with a single "class" key, and a value that is
    the class to which the majority of the datapoins in the node belong.
    
    'left' and 'right' keys are added to non-terminal nodes, which contain (possibly terminal) nodes 
    from higher levels of the tree:
    'left' corresponds to the 'low_region' key, and 'right' to the 'high_region' key
    """
    if 'low_region' in node.keys(): # not a terminal node
        lo = node['low_region']
        hi = node['high_region']
        # process left
        classes, counts = np.unique(y[lo], return_counts=True)
        if len(lo) < min_samples or current_depth == max_depth or len(classes) == 1:
            classes, counts = np.unique(y[lo], return_counts=True)
            node['left'] = {'class':classes[np.argmax(counts)]}
        else:
            node['left'] = get_split(X[lo], y[lo])
            recursive_growth(node['left'], min_samples, max_depth, current_depth + 1, X, y)

        # process right
        classes, counts = np.unique(y[hi], return_counts=True)
        if len(hi) < min_samples or current_depth == max_depth or len(classes) == 1:
            classes, counts = np.unique(y[hi], return_counts=True)
            node['right'] = {'class':classes[np.argmax(counts)]}
        else:
            node['right'] = get_split(X[hi], y[hi])
            recursive_growth(node['right'], min_samples, max_depth, current_depth + 1, X, y)

In [16]:
k = 20
test_root = get_split(X_train[:k, :], y_train[:k])
recursive_growth(test_root, 5, 10, 1, X_train[:k, :], y_train[:k])

In [17]:
def print_tree(node, depth):
    if 'class' in node.keys():
        print('.  '*(depth-1), f"[{node['class']}]")
    else:
        print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)


In [18]:
print_tree(test_root, 0)

 X_3 < 7298
.   X_37 < 1
.  .   X_5 < 50
.  .  .   X_33 < 1
.  .  .  .   X_33 < 1
.  .  .  .  .   X_33 < 1
.  .  .  .  .  .   X_33 < 1
.  .  .  .  .  .   [0]
.  .  .  .  .  .   [1]
.  .  .  .  .   [0]
.  .  .  .   [1]
.  .  .   [0]
.  .   [1]
.   [0]
 [1]


# Exercise 6 -- Make a Prediction
Use the test_node to predict the class of a compatible dataset

In [19]:
def predict_sample(node, sample):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    sample : array of size (n_features,)
        a sample datapoint
    """
    if 'class' in node.keys():
        return node['class']
    if sample[node['feature_index']] < node['tau']:
        return predict_sample(node['left'], sample)
    return predict_sample(node['right'], sample)
        
def predict(node, X):
    """
    Makes a prediction based on the decision tree defined by `node`
    
    Parameters
    ----------
    node : dictionary
        A node created one of the methods above
    X : array of size (n_samples, n_features)
        n_samples predictions will be made
    """
    prediction = np.zeros(X.shape[0])
    for i, sample in enumerate(X):
        prediction[i] = predict_sample(node, sample)
    return prediction

In [20]:
from sklearn.metrics import accuracy_score
root = get_split(X_train, y_train)
recursive_growth(root, 20, 10, 1, X_train, y_train)
test_acc = accuracy_score(y_test, predict(root, X_test))
train_acc = accuracy_score(y_train, predict(root, X_train))

print(f'Train accuracy : {train_acc}')
print(f'Test accuracy : {test_acc}')

KeyboardInterrupt: 